In [23]:
import io
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

def visualize_labeled_images(images, labels, max_outputs=3, name="image"):
    def _visualize_image(image, label):
        # Do the actual drawing in python
        fig = plt.figure(figsize=(3, 3), dpi=80)
        ax = fig.add_subplot(111)
        ax.imshow(image[::-1,...])
        ax.text(0, 0, str(label),
          horizontalalignment="left",
          verticalalignment="top")
        fig.canvas.draw()

        # Write the plot as a memory file.
        buf = io.BytesIO()
        data = fig.savefig(buf, format="png")
        buf.seek(0)

        # Read the image and convert to numpy array
        img = PIL.Image.open(buf)
        return np.array(img.getdata()).reshape(img.size[0], img.size[1], -1)

    def _visualize_images(images, labels):
        # Only display the given number of examples in the batch
        outputs = []
        for i in range(max_outputs):
            output = _visualize_image(images[i], labels[i])
            outputs.append(output)
        return np.array(outputs, dtype=np.uint8)

    # Run the python op.
    figs = tf.py_func(_visualize_images, [images, labels], tf.uint8)
    return tf.summary.image(name, figs)

In [22]:
import numpy as np
import tensorflow as tf
import uuid

def relu(inputs):
    # Define the op in python
    def _relu(x):
        return np.maximum(x, 0.)

    # Define the op's gradient in python
    def _relu_grad(x):
        return np.float32(x > 0)

    # An adapter that defines a gradient op compatible with TensorFlow
    def _relu_grad_op(op, grad):
        x = op.inputs[0]
        x_grad = grad * tf.py_func(_relu_grad, [x], tf.float32)
        return x_grad

    # Register the gradient with a unique id
    grad_name = "MyReluGrad_" + str(uuid.uuid4())
    tf.RegisterGradient(grad_name)(_relu_grad_op)

    # Override the gradient of the custom op
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": grad_name}):
        output = tf.py_func(_relu, [inputs], tf.float32)
    return output

x = tf.random_normal([10])
y = relu(x * x)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
print(sess.run(x))

with tf.Session():
    diff = tf.test.compute_gradient_error(x, [10], y, [10])
    print(diff)

[ 0.8794421   2.20649838 -0.06778084  0.39704013 -1.04980946 -0.00227016
  1.76720178  0.9822911  -0.33456507  1.28294444]
4.81605529785e-05


In [18]:
import numpy as np
import tensorflow as tf

tf.reset_default_graph()

# Placeholders are used to feed values from python to TensorFlow ops. We define
# two placeholders, one for input feature x, and one for output y.
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

# Assuming we know that the desired function is a polynomial of 2nd degree, we
# allocate a vector of size 3 to hold the coefficients. The variable will be
# automatically initialized with random noise.
w = tf.get_variable("w", shape=[3, 1])

# We define yhat to be our estimate of y.
f = tf.stack([tf.square(x), x, tf.ones_like(x)], 1)
yhat = tf.squeeze(tf.matmul(f, w), 1)

# The loss is defined to be the l2 distance between our estimate of y and its
# true value. We also added a shrinkage term, to ensure the resulting weights
# would be small.
loss = tf.nn.l2_loss(yhat - y) + 0.1 * tf.nn.l2_loss(w)

# We use the Adam optimizer with learning rate set to 0.1 to minimize the loss.
train_op = tf.train.AdamOptimizer(0.1).minimize(loss)

def generate_data():
    x_val = np.random.uniform(-10.0, 10.0, size=100)
    y_val = 5 * np.square(x_val) + 3
    return x_val, y_val

sess = tf.Session()
# Since we are using variables we first need to initialize them.
sess.run(tf.global_variables_initializer())
for _ in range(1000):
    x_val, y_val = generate_data()
    _, loss_val = sess.run([train_op, loss], {x: x_val, y: y_val})
    print(loss_val)
print(sess.run([w]))

1.698e+06
1.18267e+06
1.48117e+06
1.82511e+06
1.4511e+06
1.21819e+06
954477.0
1.21098e+06
830158.0
994920.0
1.01736e+06
1.15177e+06
717129.0
795113.0
695252.0
716860.0
774760.0
613059.0
477241.0
615982.0
496349.0
333458.0
349744.0
352581.0
332616.0
328877.0
213373.0
249426.0
236117.0
179763.0
158730.0
136573.0
135662.0
156774.0
135024.0
83963.8
76199.2
67181.3
66498.4
66240.2
52592.4
42577.1
37470.5
20062.0
24476.3
17376.7
16458.2
15182.5
10025.0
9966.22
7008.33
3530.88
3793.49
2801.26
1644.25
941.685
705.713
548.294
330.156
197.663
227.215
231.358
190.042
267.253
388.512
404.011
550.468
509.054
641.959
680.842
754.341
921.113
837.668
940.415
1108.2
749.364
935.319
1101.36
871.729
739.646
802.689
638.481
679.37
852.736
776.858
525.472
714.215
487.358
573.985
501.287
470.116
330.908
391.958
204.503
217.275
168.681
174.7
125.755
113.306
92.2854
80.327
67.8356
54.7773
44.8386
37.5437
29.7805
26.0887
22.2388
22.6473
18.1982
18.0468
19.0233
17.7504
21.0889
20.9108
20.3188
23.9678
23.8632
23

In [12]:
n = tf.constant(20)

c = tf.TensorArray(tf.int32, n)
c = c.write(0, 1)
c = c.write(1, 1)

def cond(i, a, b, c):
    return i < n

def body(i, a, b, c):
    c = c.write(i, a + b)
    return i + 1, b, a + b, c

i, a, b, c = tf.while_loop(cond, body, (2, 1, 1, c))

#print tf.Session().run(c)

d = c.stack()

print(tf.Session().run(d), tf.Session().run(c.size()))


(array([   1,    1,    2,    3,    5,    8,   13,   21,   34,   55,   89,
        144,  233,  377,  610,  987, 1597, 2584, 4181, 6765], dtype=int32), 20)
